In [11]:
import pandas as pd


In [12]:
df = pd.read_csv("small_training.csv")

In [13]:
df.head()

,prompt,responses
0,Generate abstract for the key points \r\nKey p...,The design implementation and capabilities of ...
1,Generate abstract for the key points \r\nKey p...,Information technology IT acceptance research ...
2,Generate abstract for the key points \r\nKey p...,Motivation Phylogenies are increasingly used i...
3,Generate abstract for the key points \r\nKey p...,An outstanding introduction to the fundamental...
4,Generate abstract for the key points \r\nKey p...,The technology for building knowledgebased sys...


In [14]:
# Split the data into train and test sets, with 90% in the train set
train_df = df.sample(frac=0.9, random_state=42)
test_df = df.drop(train_df.index)

# Save the dataframes to .jsonl files
train_df.to_json('train.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

In [15]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [16]:
model_name = "NousResearch/llama-2-7b-chat-hf" # use this if you have access to the official LLaMA 2 model "meta-llama/Llama-2-7b-chat-hf", though keep in mind you'll need to pass a Hugging Face key argument
dataset_name = "small_training.csv"
new_model = "llama-2-7b-custom"
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1
use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False
output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "constant"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 25
logging_steps = 5
max_seq_length = None
packing = False
device_map = {"": 0}

In [17]:
# Load datasets from CSV
train_dataset = load_dataset('csv', data_files='small_training.csv', split="train")
valid_dataset = load_dataset('csv', data_files='small_training.csv', split="train")

# Preprocess datasets
train_dataset_mapped = train_dataset.map(lambda examples: {'text': [prompt + ' ' + response for prompt, response in zip(examples['prompt'], examples['responses'])]}, batched=True)
valid_dataset_mapped = valid_dataset.map(lambda examples: {'text': [prompt + ' ' + response for prompt, response in zip(examples['prompt'], examples['responses'])]}, batched=True)


OSError: [Errno 30] Read-only file system: '/root/.cache/huggingface/datasets/_root_.cache_huggingface_datasets_csv_default-c304f0a5c531112c_0.0.0_eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d.lock'

In [ ]:
#
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="all",
    evaluation_strategy="steps",
    eval_steps=5  # Evaluate every 20 steps
)
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset_mapped,
    eval_dataset=valid_dataset_mapped,  # Pass validation dataset here
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
trainer.train()
trainer.model.save_pretrained(new_model) 



FileNotFoundError: [Errno 2] No such file or directory: '/root/.cache/huggingface/hub/models--chargoddard--llama2-22b-blocktriangular/tmp_d4202e82-c990-43c8-a56c-b4d6b1bc4a74'

In [ ]:
#Cell 4: Test the model
logging.set_verbosity(logging.CRITICAL)
prompt = f"[INST] Generate abstract for the key points\n1. Chimera Design: The design of Chimera is broken into a core and extensions. The core provides basic services and visualization, while the extensions are responsible for higher level functionality, allowing third-party developers to incorporate new features according to their needs.\n2. Multiscale Extension: The Multiscale extension of Chimera allows users to visualize large-scale molecular assemblies such as viral coats. By providing a scale-based approach, it enhances the understanding of molecular structures and interactions in biological research.\n3. Collaboratory Extension: Offering the ability for researchers based in different locations to share a Chimera session interactively, the Collaboratory extension significantly improves collaboration capacity. Through this shared environment, researchers can conduct simultaneous examinations and share insights in real-time.\n4. Other Extensions: Other extensions such as Multalign Viewer, ViewDock, Movie, and Volume Viewer offer a diverse set of features. They allow the display of multiple sequence alignments, screening of docked ligand orientations, replay of molecular dynamics trajectories, and analysis of volumetric data respectively.\n5. Real-World Usage of Chimera: The abstract also discusses the practical usage of Chimera in real-world situations, pointing out its wide applicability and impact in the field of molecular biology and bioinformatics \n . [/INST]" # replace the command here with something relevant to your task
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=1000)
result = pipe(prompt) 
print(result[0]['generated_text']) 